In [ ]:
import logging

from csv import reader
from dataclasses import dataclass
from datetime import datetime
from pathlib import Path
from typing import Any, Dict, List, Optional

In [164]:
class Template:
  pass

In [ ]:
class Employee:
    """Creates an employee object with additional functions."""

    def __init__(
        self,
        name: str,
        rank: int,
        position: str,
        hours: Dict[str, List[str]],
        lunch_hours: Dict[str, List[str]],
    ):
        """
        Initialize an `Employee` object.

        Args:
          name (str): Full name of the employee.
          rank (int): Their rank against the employee's colleagues.
          position (str): The employee's position, typically *manager*, *assistant manager*, *supervisor*, *full time*, *part time*, *shelver*, *full time security*, and *part time security*.
          hours (Dict[str, List[str]]): Dictionary mapping days to work, e.g. `{"monday": [0900, 1730]}`.
          lunch_hours (Dict[str, List[str]]): Dictionary mapping days to work, e.g. `{"monday": [1200, 1300]}`.
        """

        self.name = name
        self.first_name = self._extract_first_name(name)
        self.initials = self._generate_initials(name)

        self.rank = rank
        self.position = position

        self.hours = self._convert_to_datetime(hours)
        self.phours = self._convert_to_ptime(hours)
        self.lunch_hours = self._convert_to_datetime(lunch_hours)
        self.plunch_hours = self._convert_to_ptime(lunch_hours)

        self.programs = None

    def _extract_first_name(self, full_name: str) -> str:
        """Extract the first name from a full name string."""

        return full_name.split()[0]

    def _generate_initials(self, name: str) -> str:
        """Generate initials from a full name, handling hyphenated names."""

        return "".join(i[0] for i in name.replace("-", " ").split())

    def _convert_to_datetime(self, hours):
        """Convert string times (`"0800"`) to `datetime.time` objects."""

        result = {}
        for day, times in hours.items():
            result[day] = [datetime.strptime(t.zfill(4), "%H%M").time() for t in times]
        return result

    def _convert_to_ptime(self, hours):
        """Convert string times to readable 12-hour format (`"0800" -> "8"`)."""

        result = {}
        for day, times in hours.items():
            dt = [datetime.strptime(t.zfill(4), "%H%M") for t in times]
            hours = [str(t.hour % 12) or "12" for t in dt]
            mins = ["" if t.minute == 0 else str(t.minute) for t in dt]
            ft = [":".join([str(h), f"{m}"]).rstrip(":") for h, m in zip(hours, mins)]
            result[day] = ft
        return result

    def _get_schedule_summary(self) -> Dict[str, int]:
        """
        Get a summary of the employee's coverage.

        Returns:
          A `dict` with counts of work days and lunch days.
        """

        return {
            "work_days": len(self.hours.keys()),
            "lunch_days": len(self.lunch_hours.keys()),
        }

    def __str__(self) -> str:
        """
        Create a formatted string table of basic employee data.
        """

        name_info = f"Abbv: {self.initials}\t || First: {self.first_name}"
        pos_info = f"Rank: {self.rank:>2}\t || Position: {self.position}"

        schedule_summary = self._get_schedule_summary()
        schedule_info = f"Hrs: {schedule_summary['work_days']:>3}\t || Lunch hrs: {schedule_summary['lunch_days']}"

        lines = [name_info, pos_info, schedule_info]
        max_width = max(len(line.expandtabs()) for line in lines)

        name_padding = max(0, (max_width - len(self.name)) // 2)
        centered_name = " " * name_padding + self.name

        divider = "=" * max_width

        formatted_sections = [
            divider,
            centered_name,
            divider,
            name_info,
            pos_info,
            schedule_info,
        ]

        return "\n".join(formatted_sections) + "\n"

In [ ]:
@dataclass
class EmployeeData:
  """
  A `dataclass` to hold raw employee information before converstion to an `Employee` object.
  """

  name: str
  rank: str
  position: str
  schedule_data: Dict[str, str]

In [ ]:
class DataLoader:
  """
  Loads and parses employee data from CSV files with optional decoding of names. Does the same for template data from CSV files.
  """

  def __init__(self):
    """ Initialize the loader with empty collections for loaded data. """
    self.employees: List[Any] = []
    self.templates: List[Any] = []
    self._name_key_mapping: Optional[Dict[str, str]] = None

    self.logger = logging.getLogger(__name__)
  
  def _load_name_key_mapping(self, key_file_path: str) -> bool:
    """
    Load name key mappings from a file for decoding employee names.

    Args:
      key_file_path (str): Path to the key mapping file.
    
    Returns:
      `True` if mapping was loaded successfully, `False` otherwise.
    """

    key_path = Path(key_file_path)

    if not key_path.exists():
      self.logger.warning(f"Key mapping file not found: '{key_file_path}'")
      self._name_key_mapping = None
      return False
    
    try:
      mapping = {}
      with key_path.open("r", encoding="utf-8") as keyfile:
        for idx, line in enumerate(keyfile, 1):
          line = line.strip()

          if not line or line.startswith("#"):
            continue

          if "=" not in line:
            self.logger.warning(f"Invalid format in key file line {idx}: {line}")
            continue

          encoded, decoded = line.split("=", 1)
          mapping[encoded.strip()] = decoded.strip()
      
      self._name_key_mapping = mapping
      self.logger.info(f"Loaded {len(mapping)} name mappings from {key_file_path}")
      return True
    
    except Exception as e:
      self.logger.error(f"Failed to load key mapping from {key_file_path}: {e}")
      self._name_key_mapping = None
      return False
  
  def _decode_employee_name(self, encoded_name: str) -> str:
    """
    Decodes an employee name using the loaded key mapping.

    Args:
      encoded_name (str): The encoded name from the CSV file.
    
    Returns:
      The decoded name if mapping exists, otherwise original name.
    """

    if self._name_key_mapping is None:
      return encoded_name
    
    return self._name_key_mapping.get(encoded_name, encoded_name)
  
  def _parse_schedule_data(self, raw_schedule: Dict[str, str]) -> tuple[Dict[str, List[str]], Dict[str, List[str]]]:
    """
    Parse raw schedule strings into structured work hours and lunch hours.

    Args:
      raw_schedule (Dict[str, str]): `dict` of day keys to schedule value strings.
    
    Returns:
      `tuple` of (`work_hours_dict, lunch_hours_dict`).
    """

    work_hours = {}
    lunch_hours = {}

    for day, value in raw_schedule.items():
      if not value or value.lower() in ["off", "none", ""]:
        continue

      if "-hours" in day:
        day_name = day.replace("-hours", "")
        work_hours[day_name] = value.split("-")
      elif "-lunch" in day:
        day_name = day.replace("-lunch", "")
        lunch_hours[day] = value.split("-")
    
    return work_hours, lunch_hours
  
  def _read_csv_file(self, csv_path: Path) -> List[EmployeeData]:
    """
    Read and parse the employee CSV file into structured data objects.

    Args:
      csv_path (Path): `Path` to the employee CSV file.

    Returns:
      `List` of `EmployeeData` objects representing the parsed employees.
    """

    employee_data_list = []

    try:
      with csv_path.open("r", encoding="utf-8") as csvfile:
        csv_reader = csv.DictReader(csvfile)

        for idx, row in enumerate(csv_reader, 2):
          try:
            raw_name = row.get("name", "").strip()
            rank = row.get("rank", "").strip()
            pos = row.get("position", "").strip()

            if not all([raw_name, rank, pos]):
              self.logger.warning(f"Row {idx}: Missing required fields (name, rank, or position)")
              continue

            decoded_name = self._decode_employee_name(raw_name)

            schedule_data = {
              key: value for key, value in row.items() if ("-hours" in key or "-lunch" in key) and value
            }

            employee_data = EmployeeData(
              name=decoded_name,
              rank=rank,
              position=pos,
              schedule_data=schedule_data
            )

            employee_data.append(employee_data)
        
          except Exception as e:
            self.logger.error(f"Error processing row {row}: {e}")
            continue
    except Exception as e:
      self.logger.error(f"Failed to read CSV file at '{csv_path}': {e}")
      raise

    return employee_data_list
  
  def load_employees_from_csv(self, csv_file_path: str, key_file_path: str = "") -> Dict[str, Any]:
    """
    Load employees from a CSV file, optionally using a key mapping file.

    Args:
      csv_file_path (str): Path to the CSV file containing employee data.
    
    Returns:
      `dict` mapping employee first names to `Employee` objects.
    """

    employees_dict = {}
    csv_path = Path(csv_file_path)

    if not csv_path.exists():
      self.logger.error(f"Employee CSV file not found: '{csv_file_path}'")
      return employees_dict
    
    if key_file_path:
      mapping_loaded = self._load_name_key_mapping(key_file_path)
      if mapping_loaded:
        self.logger.info("Name key mapping loaded successfully")
      else:
        self.logger.info("Proceeding without name key mapping")
    
    try:
      employee_data_list = self._read_csv_file(csv_path)
      self.logger.info(f"Parsed {len(employee_data_list)} employee records from CSV")

      for employee_data in employee_data_list:
        try:
          work_hours, lunch_hours = self._parse_schedule_data(employee_data.schedule_data)

          employee = Employee(
            name=employee_data.name,
            rank=int(employee_data.rank),
            position=employee_data.position,
            hours=work_hours,
            lunch_hours=lunch_hours
          )

          employees_dict[employee.first_name] = employee
        except ValueError as e:
          self.logger.error(f"Invalid data for employee '{employee_data.name}': {e}")
          continue
        except Exception as e:
          self.logger.error(f"Failed to create `Employee` object for '{employee_data.name}': {e}")
          continue
      
        self.employees = list(employees_dict.values)

        self.logger.info(f"Successfully loaded {len(employees_dict)} employees")
        return employees_dict
    
    except Exception as e:
      self.logger.error(f"Critical error during loading employee: {e}")
      return employees_dict
    

In [ ]:
data_loader = DataLoader()
employees = data_loader.load_employees_from_csv("../models/employees.csv", "../models/employees_key.txt")
print(employees["Chris"])
print(employees["Jess"])
print(employees["Sonaite"])

             Chris Wright
Abbv: CW	 || First: Chris
Rank:  3	 || Position: full time
Hrs:  10	 || Lunch hrs: 8

              Jess Bryant
Abbv: JB	 || First: Jess
Rank:  3	 || Position: full time
Hrs:  10	 || Lunch hrs: 8

         Sonaite Debebe-Kumssa
Abbv: SDK	 || First: Sonaite
Rank:  2	 || Position: supervisor
Hrs:   8	 || Lunch hrs: 8



```python
import csv
import logging
from pathlib import Path
from typing import Dict, List, Optional, Any
from dataclasses import dataclass


@dataclass
class EmployeeData:
    """
    Data class to hold raw employee information before conversion to Employee object.
    
    This serves as an intermediate representation that clearly defines what data
    we expect from the CSV file, making the data flow more transparent.
    """
    name: str
    rank: str
    position: str
    schedule_data: Dict[str, str]  # Raw schedule strings from CSV


class DataLoader:
    """
    Handles loading and parsing of employee data from CSV files with optional key mapping.
    
    This class follows the single responsibility principle by focusing solely on
    data loading and conversion, while delegating object creation to appropriate factories.
    """
    
    def __init__(self):
        """Initialize the loader with empty collections for loaded data."""
        self.employees: List[Any] = []  # Will hold Employee objects
        self.templates: List[Any] = []  # Will hold Template objects  
        self._name_key_mapping: Optional[Dict[str, str]] = None
        
        # Set up logging for this class
        self.logger = logging.getLogger(__name__)

    def load_name_key_mapping(self, key_file_path: str) -> bool:
        """
        Load name key mappings from a file for decoding employee names.
        
        The key file should contain lines in the format: "encoded_name = decoded_name"
        This allows you to store encoded or abbreviated names in your CSV while
        displaying full, readable names in your application.
        
        Args:
            key_file_path: Path to the key mapping file
            
        Returns:
            True if mapping was loaded successfully, False otherwise
        """
        key_path = Path(key_file_path)
        
        if not key_path.exists():
            self.logger.warning(f"Key mapping file not found: {key_file_path}")
            self._name_key_mapping = None
            return False
        
        try:
            mapping = {}
            with key_path.open('r', encoding='utf-8') as keyfile:
                for line_number, line in enumerate(keyfile, 1):
                    line = line.strip()
                    
                    # Skip empty lines and comments
                    if not line or line.startswith('#'):
                        continue
                    
                    # Parse key-value pairs
                    if '=' not in line:
                        self.logger.warning(
                            f"Invalid format in key file line {line_number}: {line}"
                        )
                        continue
                    
                    encoded, decoded = line.split('=', 1)  # Split only on first '='
                    mapping[encoded.strip()] = decoded.strip()
            
            self._name_key_mapping = mapping
            self.logger.info(f"Loaded {len(mapping)} name mappings from {key_file_path}")
            return True
            
        except Exception as e:
            self.logger.error(f"Failed to load key mapping from {key_file_path}: {e}")
            self._name_key_mapping = None
            return False

    def _decode_employee_name(self, encoded_name: str) -> str:
        """
        Decode an employee name using the loaded key mapping.
        
        This method encapsulates the logic for name translation, making it
        easy to modify or extend the decoding behavior in the future.
        
        Args:
            encoded_name: The encoded name from the CSV file
            
        Returns:
            The decoded name if mapping exists, otherwise the original name
        """
        if self._name_key_mapping is None:
            return encoded_name
        
        return self._name_key_mapping.get(encoded_name, encoded_name)

    def _parse_schedule_data(self, raw_schedule: Dict[str, str]) -> tuple[Dict[str, List[str]], Dict[str, List[str]]]:
        """
        Parse raw schedule strings into structured work hours and lunch hours.
        
        This method separates the complex parsing logic into its own function,
        making it easier to test and modify the schedule parsing rules.
        
        Args:
            raw_schedule: Dictionary of day keys to schedule value strings
            
        Returns:
            Tuple of (work_hours_dict, lunch_hours_dict)
        """
        work_hours = {}
        lunch_hours = {}
        
        for day_key, schedule_value in raw_schedule.items():
            if not schedule_value or schedule_value.lower() in ['off', 'none', '']:
                continue
            
            if '-hours' in day_key:
                # Extract the day name by removing the '-hours' suffix
                day_name = day_key.replace('-hours', '')
                work_hours[day_name] = schedule_value.split('-')
                
            elif '-lunch' in day_key:
                # Extract the day name by removing the '-lunch' suffix  
                day_name = day_key.replace('-lunch', '')
                lunch_hours[day_name] = schedule_value.split('-')
        
        return work_hours, lunch_hours

    def _read_csv_file(self, csv_path: Path) -> List[EmployeeData]:
        """
        Read and parse the employee CSV file into structured data objects.
        
        This method focuses solely on CSV parsing and data validation,
        separating file I/O concerns from business logic.
        
        Args:
            csv_path: Path to the employee CSV file
            
        Returns:
            List of EmployeeData objects representing the parsed employees
        """
        employee_data_list = []
        
        try:
            with csv_path.open('r', encoding='utf-8') as csvfile:
                csv_reader = csv.DictReader(csvfile)
                
                for row_number, row in enumerate(csv_reader, 2):  # Start at 2 since row 1 is headers
                    try:
                        # Extract core employee information
                        raw_name = row.get('name', '').strip()
                        rank = row.get('rank', '').strip()
                        position = row.get('position', '').strip()
                        
                        # Validate required fields
                        if not all([raw_name, rank, position]):
                            self.logger.warning(
                                f"Row {row_number}: Missing required fields (name, rank, or position)"
                            )
                            continue
                        
                        # Decode the employee name if mapping is available
                        decoded_name = self._decode_employee_name(raw_name)
                        
                        # Extract all schedule-related columns
                        schedule_data = {
                            key: value for key, value in row.items() 
                            if ('-hours' in key or '-lunch' in key) and value
                        }
                        
                        # Create structured data object
                        employee_data = EmployeeData(
                            name=decoded_name,
                            rank=rank,
                            position=position,
                            schedule_data=schedule_data
                        )
                        
                        employee_data_list.append(employee_data)
                        
                    except Exception as e:
                        self.logger.error(f"Error processing row {row_number}: {e}")
                        continue  # Skip this row but continue with others
                        
        except Exception as e:
            self.logger.error(f"Failed to read CSV file {csv_path}: {e}")
            raise
        
        return employee_data_list

    def load_employees_from_csv(self, csv_file_path: str, key_file_path: str = "") -> Dict[str, Any]:
        """
        Load employees from a CSV file, optionally using a key mapping file.
        
        This is the main public method that orchestrates the entire loading process.
        It coordinates between reading the key mapping, parsing the CSV, and creating
        Employee objects, while providing comprehensive error handling and logging.
        
        Args:
            csv_file_path: Path to the CSV file containing employee data
            key_file_path: Optional path to key mapping file for name decoding
            
        Returns:
            Dictionary mapping employee first names to Employee objects
        """
        employees_dict = {}
        csv_path = Path(csv_file_path)
        
        # Validate that the CSV file exists
        if not csv_path.exists():
            self.logger.error(f"Employee CSV file not found: {csv_file_path}")
            return employees_dict
        
        # Load name key mapping if provided
        if key_file_path:
            mapping_loaded = self.load_name_key_mapping(key_file_path)
            if mapping_loaded:
                self.logger.info("Name key mapping loaded successfully")
            else:
                self.logger.info("Proceeding without name key mapping")
        
        try:
            # Parse the CSV file into structured data
            employee_data_list = self._read_csv_file(csv_path)
            self.logger.info(f"Parsed {len(employee_data_list)} employee records from CSV")
            
            # Convert parsed data into Employee objects
            for employee_data in employee_data_list:
                try:
                    # Parse the schedule data into structured format
                    work_hours, lunch_hours = self._parse_schedule_data(employee_data.schedule_data)
                    
                    # Create Employee object (assuming Employee class exists)
                    from your_employee_module import Employee  # Adjust import as needed
                    employee = Employee(
                        name=employee_data.name,
                        rank=int(employee_data.rank),  # Convert rank to integer
                        position=employee_data.position,
                        hours=work_hours,
                        lunch_hours=lunch_hours
                    )
                    
                    # Store employee using their first name as the key
                    employees_dict[employee.first_name] = employee
                    
                except ValueError as e:
                    self.logger.error(f"Invalid data for employee {employee_data.name}: {e}")
                    continue
                except Exception as e:
                    self.logger.error(f"Failed to create Employee object for {employee_data.name}: {e}")
                    continue
            
            # Store the loaded employees in the instance
            self.employees = list(employees_dict.values())
            
            self.logger.info(f"Successfully loaded {len(employees_dict)} employees")
            return employees_dict
            
        except Exception as e:
            self.logger.error(f"Critical error during employee loading: {e}")
            return employees_dict

    def get_loading_summary(self) -> Dict[str, int]:
        """
        Get a summary of the current loading state.
        
        This method provides visibility into what has been loaded, which is
        useful for debugging and monitoring the application state.
        
        Returns:
            Dictionary with counts of loaded objects and mapping status
        """
        return {
            'employees_loaded': len(self.employees),
            'templates_loaded': len(self.templates),
            'has_name_mapping': self._name_key_mapping is not None,
            'name_mappings_count': len(self._name_key_mapping) if self._name_key_mapping else 0
        }
```